# Updating JSON Documents
Updated: 2019-10-03

## Document Maintenance
The current ISO SQL JSON standard does not provide any definition for an SQL function to update or delete objects or values within a JSON document. From the ISO perspective, the only way to update a JSON document is to extract it from the database, modify it with an application, and then replace the entire document back into a table. This does not help a DBA or developer easily make quick fixes to an individual document or apply table-wide changes to existing documents.

The Db2 JSON SYSTOOLS functions that were developed as a hidden part of Db2's NoSQL API support (based on the MongoDB wire protocol) were documented in Db2 Version 11.1.2.2 under the "SQL access to JSON documents" section and were also added to the system catalogs at that time. While these functions do not conform to the ISO SQL JSON standard, they do provide some functionality that is currently not available with the new ISO JSON functions with the one of specific interest to us in this notebook being the `JSON_UPDATE` function.

**Note:** To use the SYSTOOLS `JSON_UPDATE` function, you must store the data as BSON in a BLOB column.

### JSON_UPDATE
The `JSON_UPDATE` function is part of the SYSTOOLS schema, so it requires the user or application be granted EXECUTE privilege on the function as well as either explicitly qualify any reference to the function with the SYSTOOLS schema (or add SYSTOOLS to the `CURRENT PATH` special register). 

The syntax of the JSON_UPDATE function is:
```sql
   JSON_UPDATE(document, '{$set  : {field:value}}')
                         '{$unset: {field:null}}'
```

The arguments are:
* document – BSON document
* action – the action we want to take which consists of:
    - operation (`$set` or `$unset`)
    - key – The key we are looking for
    - value – The value we want to set the field to

There are three possible outcomes from using the `JSON_UPDATE` statement:
* If the field is found, the existing value is replaced with the new one when the `$set` is specified
* If the field is not found, the field:value pair is added to the document when `$set` is specified
* If you use the `$unset` keyword and set the value to `null`, the field is removed from the document

There are some significant differences between the arguments used with `JSON_UPDATE` compared to the ISO SQL JSON functions. The first difference is that the document must be in BSON format. This excludes direct access to any JSON documents that you may have stored as character strings. In addition, if a table column is used as the source, the data type of the column must be BLOB.

The field that defines the JSON object also has a different format. With the new ISO JSON functions, you can specify a path to find the target without the root character used by those JSON functions (I.e. `$.`).

You can always convert your documents to BSON format using the new `JSON_TO_BSON` conversion function (and restore them to JSON afterwards) if you find the `JSON_UPDATE` function to be useful. See the example at the end of this section.

### Adding or Updating a New Key-Value Pair
We will create a BOOK table contains a JSON document with the following information.

**Note:** To represent `null` in a Python dictionary (JSON), the value `None` is used instead.

In [ ]:
book_info = \
{
 "authors": {
            "primary"   : {"first_name":"Paul",  "last_name":"Bird"},
            "secondary" : {"first_name":"George","last_name":"Baklarz"}       
            },
 "foreword": {
            "primary"   : {"first_name":"Thomas","last_name":"Hronis"}
            },
 "formats": ["Hardcover","Paperback",None,"PDF"]
}

### Load Db2 Extensions
The Db2 Jupyter extensions need to be loaded in order to run any of the examples in this notebook. In addition, a `CONNECT` command needs to be issued to connect to the local Db2 database. The default `SAMPLE` database is assumed to exist on the local system. If not, you need to modify the `CONNECT` command to use the appropriate userid, database, and host parameters.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

To simplify updating of JSON values, the table should be defined with a BLOB column. 

In [ ]:
%%sql
DROP TABLE BOOKS;
CREATE TABLE BOOKS(INFO BLOB(2000) INLINE LENGTH 2000);

Any inserts into this table would need to ensure that the data is converted to BSON format.

In [ ]:
%sql INSERT INTO BOOKS VALUES JSON_TO_BSON(:book_info)

You can use the `JSON_UPDATE` function with regular JSON (character) data, but you will first need to convert this data to BSON, execute the UPDATE statement, and then convert it back to character JSON.

To add a new field to the record, the `JSON_UPDATE` function needs to specify the target key and the new or replacement value, including the full nesting within the document. Since there is no existing key that matches, the following SQL will add a new field called *publish_date* with the date that the book was made available.

In [ ]:
%%sql
UPDATE BOOKS
  SET INFO = SYSTOOLS.JSON_UPDATE(INFO,
            '{ $set: {"publish_date": "2018-12-31"}}');

The resulting document now contains the new field.

In [ ]:
%sql -j SELECT BSON_TO_JSON(INFO) FROM BOOKS;

If the publish_date field already existed, then the current value for the key would have been replaced by the new value. In the following example, the `JSON_UPDATE` function would replace the date with the new value.

In [ ]:
%%sql -j
UPDATE BOOKS
  SET INFO = SYSTOOLS.JSON_UPDATE(INFO,
             '{ $set: {"publish_date": "2018-11-30"}}');
SELECT BSON_TO_JSON(INFO) FROM BOOKS;

To update a column that contained character-based JSON, you would need to add appropriate functions that convert the data to and from BSON in order for the update to work.
```sql
UPDATE BOOKS
  SET INFO = 
    BSON_TO_JSON(
       SYSTOOLS.JSON_UPDATE(
                            JSON_TO_BSON(INFO),
                              '{ $set: {"publish_date": "2018-12-31"}}'
                           )
                );
```

### Adding or Updating a New Array Value
Adding a new value to an array requires some care. The formats field contains four different ways that a book is available for reading. If we want to add a new format (Audio Book), it would be tempting to use the same syntax that was used for adding a new publish date.

In [ ]:
%%sql 
UPDATE BOOKS
  SET INFO = 
    SYSTOOLS.JSON_UPDATE(INFO,'{ $set: {"formats": "Audio Book"}}');

Unfortunately, this ends up wiping out the array and replacing it with just a single value: 

In [ ]:
%sql -j SELECT JSON_QUERY(INFO,'$.formats') FROM BOOKS

Using the array specification would seem to be a better approach, but the `JSON_UPDATE` function does not use the ISO SQL JSON path method of referring to an array item. To refer to an element in an array, you must append a dot (`.`) after the array name followed by the array index value. So rather than specifying `formats[0]`, the path would be `formats.0`.

We reset the book in our table and then perform the update. This SQL will replace element zero of the array (Hardcover) with "Audio Book".

In [ ]:
%%sql
DELETE FROM BOOKS;
INSERT INTO BOOKS VALUES JSON_TO_BSON(:book_info);
UPDATE BOOKS 
  SET INFO = 
    SYSTOOLS.JSON_UPDATE(INFO,'{ $set: {"formats.0": "Audio Book"}}');

We check the contents of the array to see what the contents are.

In [ ]:
%sql -j SELECT JSON_QUERY(INFO,'$.formats') FROM BOOKS

The only way to insert a new value into the array is to pick an index value that is greater than what the list could possibly be. If we reset the table back to the original value that we started with and then issue the following SQL, the formats field will contain the new value.

In [ ]:
%%sql -j
DELETE FROM BOOKS;
INSERT INTO BOOKS VALUES JSON_TO_BSON(:book_info);
UPDATE BOOKS 
  SET INFO = 
    SYSTOOLS.JSON_UPDATE(INFO,'{ $set: {"formats.999": "Audio Book"}}');
SELECT JSON_QUERY(INFO,'$.formats') FROM BOOKS   

Note that the new array element will be placed at the end as the specified index is 999 which is greater than the current size of the array, but the new element will have the array index value of 4 (JSON arrays start at index 0!) not the 999 specified in the `JSON_UPDATE` call.

### Removing a Field
To remove a field from a document you must use the following syntax:
```sql
   JSON_UPDATE(document, '{$unset: {field:null}}')
```

The field must be set to `null` to remove it from the document and the operation is now `$unset` (not the `$set` we used before). Our modified BOOKS table contains the publish_date which now will be removed.

In [ ]:
%%sql -j
UPDATE BOOKS
  SET INFO = 
    SYSTOOLS.JSON_UPDATE(INFO,'{ $unset: {"publish_date": null}}');
SELECT BSON_TO_JSON(INFO) FROM BOOKS;

It is not actually possible to remove an item from an array, but it is possible to set the specific array value to `null`. Again, you must use the SYSTOOLS functions approach to array specification instead of the JSON SQL path expression we have discussed in previous chapters.

This SQL will set the "Audio Books" array item to null in the list but will not actually remove it. Here we have to specify the specific array index value that we want to remove (which is 4).

In [ ]:
%%sql -j
UPDATE BOOKS
  SET INFO = 
    SYSTOOLS.JSON_UPDATE(INFO,'{ $unset: {"formats.4": null}}');
SELECT JSON_QUERY(INFO,'$.formats') FROM BOOKS;

You can't remove the null value from the array. `JSON_UPDATE` does not remove the actual array entry when a delete occurs in order that the index values for subsequent elements within an array will be preserved.  Although, in this example there are no entries after the one affected, `JSON_UPDATE` does not try to be too clever about this, it just does not remove them.

### Updating JSON documents stored as Character Strings
The `JSON_UPDATE` function requires that the document be stored as a BSON object in a BLOB column. If your documents are currently stored as character string, then you will need to add some additional logic around the `UPDATE` statement.

The BOOKS table was recreated in the following format.

In [ ]:
%%sql
DROP TABLE BOOKS;
CREATE TABLE BOOKS(INFO VARCHAR(2000));
INSERT INTO BOOKS VALUES (:book_info);

To add the new publish_date field to the record, we would use the following `UPDATE` statement.

In [ ]:
%%sql
UPDATE BOOKS
  SET INFO =
    BSON_TO_JSON( 
                 SYSTOOLS.JSON_UPDATE(JSON_TO_BSON(INFO),
                   '{ $set: {"formats.999": "Audio Book"}}')
                );

As of Db2 11.1.4.4, the JSON SYSTOOLS functions are compatible with the BSON storage format used by the ISO SQL JSON functions so that is why the `BSON_TO_JSON` and `JSON_TO_BSON` functions are used rather than the original SYSTOOLS conversion functions.

We check the contents of the book document to make sure our Audio book has been added.

In [ ]:
%%sql -j
SELECT JSON_QUERY(INFO,'$.formats') FROM BOOKS

## Summary
The ISO SQL JSON functions currently do not provide a mechanism for adding, updating, or deleting objects or elements within a JSON document. Without this capability, applications will need to retrieve entire documents, modify them, and then re-store them back into the database.

Db2 includes a JSON SYSTOOLS function called `JSON_UPDATE` that allows for the update of key:value pairs within a JSON document. It has some restrictions on the format that the document must be in and uses a slightly different JSON path expression that the standard uses. However, in situations where simple updates or quick fixes are required, this function may be sufficient. The only drawback is that this function is not part of the ISO SQL standard and may be discontinued at a future date once a replacement is made available.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]